In [1]:
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from utils import read_lastfm
from gensim.models import Word2Vec

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from sklearn.dummy import DummyClassifier

# Song2vec loading

In [2]:
def build_vocab(model):
    emb_vectors = {}
    for n in model.wv.index_to_key:
        emb_vectors[n] = model.wv[n]
    return emb_vectors

def load_model(filename):
    model = Word2Vec.load(filename)
    emb_vectors = build_vocab(model)
    return emb_vectors, model

In [3]:
emb_vectors, model = load_model("data/word2vec/word2vec.model")
s2v_df = pd.DataFrame(data=emb_vectors.values(), index=emb_vectors.keys())

# TF-IDF initialization

In [4]:
songs, users = read_lastfm(zip_name="data/lastfm-dataset-1K.zip")

In [5]:
songs["song_id"]= songs.artist_name.cat.codes.astype("int64") * songs.track_name.nunique() \
                            + songs.track_name.cat.codes

In [6]:
corpus_df = songs.sort_values(["user_id", "timestamp"]).groupby("user_id")\
                .agg(sequence=("song_id", list))

In [7]:
corpus = corpus_df.values[:,0]

In [8]:
vectorizer = TfidfVectorizer(tokenizer=lambda x: x, lowercase=False)
X = vectorizer.fit_transform(corpus)

In [9]:
X.shape

(992, 1498727)

In [10]:
def get_song_vector(song_id, X, vectorizer):
    idx = vectorizer.vocabulary_[song_id]
    return X[:, idx].toarray()[:,0]

In [11]:
get_song_vector(3583077562, X, vectorizer)[:10]

array([0.01709776, 0.        , 0.        , 0.02194015, 0.        ,
       0.00360557, 0.        , 0.        , 0.        , 0.        ])

In [12]:
song_ids = list(vectorizer.vocabulary_.keys())
indices = list(vectorizer.vocabulary_.values())

In [13]:
song_ids = [x for _,x in sorted(zip(indices,song_ids))]

In [14]:
tfidf_df = pd.DataFrame(data=X.T.todense(), index=song_ids)

# Classification Tasks

* Predict if two songs appear in the same context
* Predict if two songs are from the same artist
* Predict the tag of a song

In [15]:
SEED = 42

In [16]:
def compute_metrics(y_test, y_pred, y_proba):
    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_proba)
    return acc, f1, auc

In [39]:
def predict_RF(X_train, X_test, y_train, y_test, 
               param_grid = {'n_estimators': [2, 50 ,100, 150],
                      'criterion': ['gini', 'entropy']}):    
    model = RandomForestClassifier(random_state=SEED)
    cv = KFold(n_splits=5, shuffle=True, random_state=SEED)
    print("Starting Grid search")
    grid = GridSearchCV(model, param_grid, cv=cv, scoring='f1', verbose=4)
    grid.fit(X_train, y_train)
    best_model = grid.best_estimator_
    print(f"Best parameters are: {grid.best_params_}")
    y_pred = best_model.predict(X_test)
    y_proba = best_model.predict_proba(X_test)[:,1]
    return compute_metrics(y_test, y_pred, y_proba)

In [18]:
def predict_random(X_train, y_train, X_test, y_test):
    random = DummyClassifier(strategy='uniform', random_state=SEED)
    random.fit(X_train, y_train)
    y_pred = random.predict(X_test)
    y_proba = random.predict_proba(X_test)[:, 1]
    return compute_metrics(y_test, y_pred, y_proba)

def predict_majority(X_train, y_train, X_test, y_test):
    majority = DummyClassifier(strategy='most_frequent', random_state=SEED)
    majority.fit(X_train, y_train)
    y_pred = majority.predict(X_test)
    y_proba = majority.predict_proba(X_test)[:, 1]
    return compute_metrics(y_test, y_pred, y_proba)

In [19]:
def create_vectors_pairs(vectors, song_pairs):
    vector_pairs = vectors.merge(song_pairs, right_on="song1", left_index=True)\
            .merge(vectors, left_on="song2", right_index=True)\
            .drop(['song1', 'song2'], axis=1)
    print(len(vector_pairs))
    print(len(song_pairs))
    assert len(vector_pairs) == len(song_pairs)
    assert vector_pairs.shape[1] == vectors.shape[1] * 2
    return vector_pairs

## Same context classification

In [47]:
sorted_songs = songs.sort_values(["user_id", "timestamp"])

In [48]:
def pick_song_in_same_context(songs, half_n):
    """Simply chooses random songs pair them with the previous or following one"""
    np.random.seed(SEED)
    idx1 = np.random.randint(1, len(songs) - 1, half_n) #Exclude first and last song
    idx2 = idx1.copy() #Offset the first index by -1 or 1 
    idx2[::2] += 1
    idx2[1::2] -= 1
    songs1 = songs.iloc[idx1].song_id.values
    songs2 = songs.iloc[idx2].song_id.values
    X = np.c_[songs1, songs2]
    y = np.ones(half_n)
    return X, y
    
def pick_songs_in_diff_context(songs, half_n):
    songs1 = songs.sample(half_n, random_state=SEED).song_id.values
    songs2 = songs.sample(half_n, random_state=SEED + 1).song_id.values
    X = np.c_[songs1, songs2]
    y = np.zeros(half_n)
    return X, y

def create_context_dataset(sorted_songs, n=20000):
    """Create a dataset of song pairs that either appeared in the same context or not""" 
    X_pos, y_pos = pick_song_in_same_context(sorted_songs, n//2)
    X_neg, y_neg = pick_songs_in_diff_context(sorted_songs, n//2)
    old_X = np.r_[X_pos, X_neg]
    old_y = np.r_[y_pos, y_neg]
    dataset = list(zip(old_X, old_y))
    np.random.seed(SEED)
    np.random.shuffle(dataset)
    X, y = zip(*dataset)
    return pd.DataFrame(X, columns=["song1", "song2"]), np.array(y)

In [49]:
song_pairs, labels = create_context_dataset(sorted_songs, n= 10000)

In [50]:
#Only keep song with s2v embeddings
song_pairs = song_pairs.copy()
song_pairs['labels'] = labels
song_pairs = song_pairs[song_pairs.song1.isin(s2v_df.index) & song_pairs.song2.isin(s2v_df.index)]
labels = song_pairs.labels
song_pairs.drop('labels', axis=1, inplace=True)

### TF-IDF

In [59]:
tfidf_pairs = create_vectors_pairs(tfidf_df, song_pairs)

8485
8485


In [60]:
tfidf_pairs.head()

,0_x,1_x,2_x,3_x,4_x,5_x,6_x,7_x,8_x,9_x,...,982_y,983_y,984_y,985_y,986_y,987_y,988_y,989_y,990_y,991_y
1382,0.0,0.000000,0.0,0.009143,0.0,0.0,0.0,0.0,0.0,0.0,...,0.003974,0.0,0.0,0.002027,0.0,0.0,0.0,0.0,0.0,0.0
6851,0.0,0.000000,0.0,0.005735,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
8584,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3106,0.0,0.000804,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
9648,0.0,0.000790,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


In [61]:
X_train, X_test, y_train, y_test = \
                train_test_split(tfidf_pairs, labels, train_size=0.8, random_state=SEED)
acc, f1, auc = predict_RF(X_train, X_test, y_train, y_test)

Starting Grid search
Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV 1/5] END .................criterion=gini, n_estimators=2; total time=   0.5s
[CV 2/5] END .................criterion=gini, n_estimators=2; total time=   0.4s
[CV 3/5] END .................criterion=gini, n_estimators=2; total time=   0.3s
[CV 4/5] END .................criterion=gini, n_estimators=2; total time=   0.4s
[CV 5/5] END .................criterion=gini, n_estimators=2; total time=   0.4s
[CV 1/5] END ................criterion=gini, n_estimators=50; total time=   5.0s
[CV 2/5] END ................criterion=gini, n_estimators=50; total time=   4.5s
[CV 3/5] END ................criterion=gini, n_estimators=50; total time=   4.3s
[CV 4/5] END ................criterion=gini, n_estimators=50; total time=   4.2s
[CV 5/5] END ................criterion=gini, n_estimators=50; total time=   4.4s
[CV 1/5] END ...............criterion=gini, n_estimators=100; total time=   8.3s
[CV 2/5] END ...............

In [62]:
print("TF-IDF:", acc, f1, auc)
print("Random:", predict_random(X_train, y_train, X_test, y_test))
print("Majority:",predict_majority(X_train, y_train, X_test, y_test))

TF-IDF: 0.5173836181496759 0.5179517363154797 0.501126708122857
Random: (0.49616971125515613, 0.48955223880597015, 0.5)
Majority: (0.4926340601060695, 0.6600868535333597, 0.5)


### song2vec

In [55]:
s2v_pairs = create_vectors_pairs(s2v_df, song_pairs)

8485
8485


In [56]:
s2v_pairs.head()

,0_x,1_x,2_x,3_x,4_x,5_x,6_x,7_x,8_x,9_x,...,90_y,91_y,92_y,93_y,94_y,95_y,96_y,97_y,98_y,99_y
2416,0.050536,0.700072,1.502735,0.799808,1.421927,-4.074349,-0.018887,1.857755,-2.008438,-0.825392,...,2.049515,1.042242,-2.304445,-0.645610,1.034146,0.522461,-0.801188,-2.008663,-0.396661,0.024584
3249,-0.932494,1.682110,-0.600307,1.201052,1.746724,0.416302,-0.009236,0.224624,0.714160,1.807721,...,2.049515,1.042242,-2.304445,-0.645610,1.034146,0.522461,-0.801188,-2.008663,-0.396661,0.024584
2532,0.050536,0.700072,1.502735,0.799808,1.421927,-4.074349,-0.018887,1.857755,-2.008438,-0.825392,...,1.072537,-0.068162,-1.121787,-0.130818,-0.168337,-0.234738,-0.270714,-0.263553,-0.453970,-0.143811
1651,-0.885090,3.123382,0.424417,1.275812,1.166315,-1.786570,-1.911305,2.292119,-1.925104,1.388782,...,0.514146,-0.032140,-0.602289,-0.052574,-0.092585,-0.073549,-0.129805,-0.187154,-0.227991,-0.060000
4873,-0.885090,3.123382,0.424417,1.275812,1.166315,-1.786570,-1.911305,2.292119,-1.925104,1.388782,...,1.174746,0.092594,-3.189829,0.784431,-0.239225,1.520638,0.963496,-1.337876,-1.011305,0.964345


In [57]:
X_train, X_test, y_train, y_test = \
                train_test_split(s2v_pairs, labels, train_size=0.8, random_state=SEED)
acc, f1, auc = predict_RF(X_train, X_test, y_train, y_test)

Starting Grid search
Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV 1/5] END .................criterion=gini, n_estimators=2; total time=   0.2s
[CV 2/5] END .................criterion=gini, n_estimators=2; total time=   0.2s
[CV 3/5] END .................criterion=gini, n_estimators=2; total time=   0.2s
[CV 4/5] END .................criterion=gini, n_estimators=2; total time=   0.2s
[CV 5/5] END .................criterion=gini, n_estimators=2; total time=   0.2s
[CV 1/5] END ................criterion=gini, n_estimators=50; total time=   3.8s
[CV 2/5] END ................criterion=gini, n_estimators=50; total time=   3.6s
[CV 3/5] END ................criterion=gini, n_estimators=50; total time=   3.9s
[CV 4/5] END ................criterion=gini, n_estimators=50; total time=   3.7s
[CV 5/5] END ................criterion=gini, n_estimators=50; total time=   3.8s
[CV 1/5] END ...............criterion=gini, n_estimators=100; total time=   7.4s
[CV 2/5] END ...............

In [58]:
print("Song2Vec:", acc, f1, auc)

Song2Vec: 0.5085444902769594 0.5250569476082005 0.49960822232966007


## Same artist classification

In [26]:
discography = songs[["artist_name", "track_name", "song_id"]].drop_duplicates()

In [27]:
def pick_songs_from_same_artist(discography, half_n):
    """Simply chooses random songs pair them with the previous or following one"""
    artist_with_multiple_songs = discography.groupby("artist_name")\
                                .agg(count=("track_name", "count")).reset_index()
#     print(len(discography.merge(tfidf_df, left_on="song_id", right_index=True)))
    discography = discography.merge(artist_with_multiple_songs)
    
    song_pairs = discography.groupby("artist_name").agg(song1=("song_id", "first"), 
                                                        song2=("song_id", "last"))
    X = song_pairs.sample(half_n, random_state=SEED).values
#     print(len(X))
#     print(len(tfidf_df.index.isin(X_pos[:,0])))
    y = np.ones(half_n)
    return X, y
    
def pick_songs_from_diff_artist(discography, half_n):
    discography = discography.drop_duplicates("artist_name")
    songs = discography.sample(2*half_n, random_state=SEED).song_id.values
    songs1 = songs[:half_n]
    songs2 = songs[half_n:]
    X = np.c_[songs1, songs2]
    y = np.zeros(half_n)
    return X, y

def create_artist_dataset(discography, n=20000):
    """Create a dataset of song pairs that either appeared in the same context or not""" 
    discography = discography.merge(s2v_df, left_on="song_id", right_index=True)
    discography = discography.merge(tfidf_df, left_on="song_id", right_index=True)
    X_pos, y_pos = pick_songs_from_same_artist(discography, n//2)
    X_neg, y_neg = pick_songs_from_diff_artist(discography, n//2)
#     print(len(tfidf_df))
#     print(sum(tfidf_df.index.isin(X_pos[:,0])))
#     print(sum(tfidf_df.index.isin(X_pos[:,1])))
    old_X = np.r_[X_pos, X_neg]
    old_y = np.r_[y_pos, y_neg]
    dataset = list(zip(old_X, old_y))
    np.random.seed(SEED)
    np.random.shuffle(dataset)
    X, y = zip(*dataset)
    return pd.DataFrame(X, columns=["song1", "song2"]), np.array(y)

In [ ]:
song_pairs, labels = create_artist_dataset(discography, 20000)

In [ ]:
#Only keep song with s2v embeddings
song_pairs = song_pairs.copy()
song_pairs['labels'] = labels
song_pairs = song_pairs[song_pairs.song1.isin(s2v_df.index) & song_pairs.song2.isin(s2v_df.index)]
song_pairs = song_pairs[song_pairs.song1.isin(tfidf_df.index) & song_pairs.song2.isin(tfidf_df.index)]
labels = song_pairs.labels
# song_pairs.drop('labels', axis=1, inplace=True)

In [76]:
print(len(labels), sum(labels))

13547 3547.0


### TF-IDF

In [70]:
tfidf_pairs = create_vectors_pairs(tfidf_df, song_pairs)

6774
6774


In [71]:
X_train, X_test, y_train, y_test = \
                train_test_split(tfidf_pairs, labels, train_size=0.8, random_state=SEED)

In [41]:
X_train, X_test, y_train, y_test = \
                train_test_split(tfidf_pairs, labels, train_size=0.8, random_state=SEED)
acc, f1, auc = predict_RF(X_train, X_test, y_train, y_test)

Starting Grid search
Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV 1/5] END .................criterion=gini, n_estimators=2; total time=   0.4s
[CV 2/5] END .................criterion=gini, n_estimators=2; total time=   0.4s
[CV 3/5] END .................criterion=gini, n_estimators=2; total time=   0.4s
[CV 4/5] END .................criterion=gini, n_estimators=2; total time=   0.3s
[CV 5/5] END .................criterion=gini, n_estimators=2; total time=   0.4s
[CV 1/5] END ................criterion=gini, n_estimators=50; total time=   7.8s
[CV 2/5] END ................criterion=gini, n_estimators=50; total time=   6.7s
[CV 3/5] END ................criterion=gini, n_estimators=50; total time=   7.6s
[CV 4/5] END ................criterion=gini, n_estimators=50; total time=   9.2s
[CV 5/5] END ................criterion=gini, n_estimators=50; total time=   7.8s
[CV 1/5] END ...............criterion=gini, n_estimators=100; total time=  13.6s
[CV 2/5] END ...............

In [42]:
print("TF-IDF:", acc, f1, auc)

TF-IDF: 0.6546125461254613 0.13333333333333333 0.4766000294570833


### Song2vec

In [43]:
s2v_pairs = create_vectors_pairs(s2v_df, song_pairs)

6774
6774


In [44]:
X_train, X_test, y_train, y_test = \
                train_test_split(s2v_pairs, labels, train_size=0.8, random_state=SEED)
acc, f1, auc = predict_RF(X_train, X_test, y_train, y_test)

Starting Grid search
Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV 1/5] END .................criterion=gini, n_estimators=2; total time=   0.1s
[CV 2/5] END .................criterion=gini, n_estimators=2; total time=   0.2s
[CV 3/5] END .................criterion=gini, n_estimators=2; total time=   0.1s
[CV 4/5] END .................criterion=gini, n_estimators=2; total time=   0.1s
[CV 5/5] END .................criterion=gini, n_estimators=2; total time=   0.2s
[CV 1/5] END ................criterion=gini, n_estimators=50; total time=   3.1s
[CV 2/5] END ................criterion=gini, n_estimators=50; total time=   3.3s
[CV 3/5] END ................criterion=gini, n_estimators=50; total time=   3.3s
[CV 4/5] END ................criterion=gini, n_estimators=50; total time=   3.6s
[CV 5/5] END ................criterion=gini, n_estimators=50; total time=   3.3s
[CV 1/5] END ...............criterion=gini, n_estimators=100; total time=   6.1s
[CV 2/5] END ...............

In [45]:
print("Song2Vec:", acc, f1, auc)

Song2Vec: 0.7070110701107011 0.13507625272331156 0.5006755302274993


In [46]:
print("Random:", predict_random(X_train, y_train, X_test, y_test))
print("Majority:",predict_majority(X_train, y_train, X_test, y_test))

Random: (0.5003690036900369, 0.34079844206426485, 0.5)
Majority: (0.7402214022140221, 0.0, 0.5)
